In [4]:
import os, shutil, xml, zipfile, pandas, glob
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import xmltodict 

hwp_file = r'../data/template1.hwpx'
temp_dir = r'../temp'

display(
    os.path.splitext(hwp_file),
    os.path.splitext(os.path.basename(hwp_file))[0],
    temp_base_name := os.path.join(temp_dir, os.path.splitext(os.path.basename(hwp_file))[0]),
    temp_zip_file := f'{temp_base_name}.zip',
    temp_unzip_dir := temp_base_name,
    target_xml_file := os.path.join(temp_base_name, 'Contents', 'section0.xml')
)

if not os.path.exists(temp_zip_file):
    shutil.copy(hwp_file, temp_zip_file)

with zipfile.ZipFile(temp_zip_file, 'r') as zf:
    zf.extractall(path=temp_base_name)

with open(target_xml_file, 'r') as file:
    xml_doc = file.read()

soup = BeautifulSoup(xml_doc, 'lxml')


('../data/template1', '.hwpx')

'template1'

'../temp/template1'

'../temp/template1.zip'

'../temp/template1'

'../temp/template1/Contents/section0.xml'

In [34]:
_dict = {
    '보고일': None,
    '보고자': None,
    '평가일': None,
    '평가자': None,
    '병록번호': None,
    '환자명': None,
    '나이': None,
    '유해사례 내용': None,
    '약사평가': None,
    '참고문헌': None,
    # 'WHO-UMC causality assessment': None,
    # 'Naranjo Scale': None,
    # '한국형 Algorithm Ver 2.0': None
}

In [50]:
keys = _dict.keys()
tags = soup.select('hp\:t')
l, r = 0, 0
cnt = 100    

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9]'
    return re.sub(pattern, '', text)

while r < len(tags):
    key = _dict.keys()
    tag = tags[r]
    
    if tag.string in key:
        target_key = tag.string
        r += 1

        target_value = ''
        
        while not tags[r].string:
            r += 1
        
        print(f'key: {target_key} \t {tags[r].string}')

    else:
        r += 1
            

key: 보고일 	  : 2000-01-01
key: 보고자 	  : 홍길동
key: 병록번호 	  : 0000****
key: 환자명 	  : 김**
key: 나이 	  : 01
key: 유해사례 내용 	 (신경정신) 두통, (근골격계) 근육통


In [44]:
import re

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9]'
    return re.sub(pattern, '', text)

# Example usage
text = "Hello @World!123"
clean_text = remove_special_characters(text)
print(clean_text)

Hello @World123
